# 악기 브랜드(Gibson) 사이트 크롤링 과제
## 1. 목표: 깁슨 사이트에서 제공하는 모든 악기의 데이터를 수집한다. 
## 2. 자료 수집: 이름, 하이퍼링크, 가격, 이미지, 상세스펙
## 3. CSV파일로 저장

In [2]:
# 사용할 모듈
import requests # url로 요청을 보내기 위해
from bs4 import BeautifulSoup # 이쁘게 만들기 위해
from pprint import pprint # 이쁘게 출력하기 위해
import os # 운영체제에서 제공되는 기본기능을 사용하기 위해
import pandas as pd # 데이터 프레임 만들기 위해 
        
# 모듈 명령어
# dir(Gibson_Crawling) 으로 호출
intro = """Les-Paul, SG, ES, Acoustic, Designer, Bass 를 'crl.run( )'안에 입력하세요.\n입력한 값에 대한 깁슨 홈페이지에 있는 모든 깁슨 모델을 제공합니다."""
maker = 'jhkang1517@gmail.com'

BASE_DIR = os.getcwd() # os명령어. get current work directory 현재 작업 주소
# BASE_DIR

class crl():
    def __init__(self, base_dir):
        self.base_dir = BASE_DIR
    
    def run(model):         
        PICTURE_DIR = os.path.join(BASE_DIR, 'guitar_pictures_{}'.format(model))
        # Ex. 폴더명: guitar_picture_Acoustic
        if not os.path.exists(PICTURE_DIR):
            os.makedirs(PICTURE_DIR) # 만약 경로가 없으면, 새로운 폴더를 생성하겠습니다.
    
        guitar_list = [] # 새로운 빈 리스트를 생성합니다.
        gibson_usl = "https://www.gibson.com/Guitars/" + model # 파라미터 보다 쉽다.
        # params = {
        #     'Guitars': 'Guitars',
        #     'Les-Paul': 'Les-Paul'
        # }
        # gibson_usl
        
        resp = requests.get(gibson_usl) # 요청보내기
        # resp
        # resp.url
        # resp.content
        
        soup = BeautifulSoup(resp.content, 'html.parser') # Soup 만들기
        # soup
            
        gibson_contents = soup.find('div', class_='row pb-5').find_all('div', class_='col-sm-12 col-md-6 col-lg-4 px-2 mb-4')
        # gibson_contents # 가져올 웹사이트 내 컨텐츠 범위 결정

        for div in gibson_contents:
            guitar_dict = {}

            a_tag = div.find('h4').find('a')
            # print(a_tag)
            guitar_dict['link'] = 'https://www.gibson.com'+a_tag['href'] # 링크
            guitar_dict['name'] = a_tag.text # 악기 이름
            # print(a_tag['href'])
    
            b_tag = div.find('div', class_='price-label ml-3').find('span')
            guitar_dict['price'] = b_tag.text.strip() #악기 가격
            # print(b_tag)
            
            # 상세페이지 내용 가져오기 
            detail_resp = requests.get('https://www.gibson.com'+a_tag['href'])
            # detail_resp
            # detail_resp.url
            # detail_resp.content
            
            detail_soup = BeautifulSoup(detail_resp.content, 'html.parser')
            # detail_soup
            
            detail_content = detail_soup.find('div', class_='gibson-card py-4')
            # detail_content
    
            d_tag = detail_content.find('div', class_='spec-section body-specs pb-4 mb-4')
            e_tag = detail_content.find('div', class_='spec-section neck-specs mb-4')
            f_tag = detail_content.find('div', class_='spec-section hardware-specs mb-4')
            g_tag = detail_content.find('div', class_='spec-section electronics-specs mb-4')
            h_tag = detail_content.find('div', class_='spec-section miscellaneous-specs mb-4')

            
# \nBody\n\nBody Material\nSwamp Ash\n\n\nWeight Relief\nNone\n\n\nFinish\nSatin Nitrocellulose Lacquer\n\n
            guitar_dict['Body'] = d_tag.text.strip('\nBody\n\n ').split("\n\n\n")
            guitar_dict['Neck'] = e_tag.text.strip('\nNeck\n\n ').split("\n\n\n")
            guitar_dict['Hardware'] = f_tag.text.strip('\nHardware\n\n ').split("\n\n\n")
            guitar_dict['Electronics'] = g_tag.text.strip('\nElectronics\n\n ').split("\n\n\n")
            guitar_dict['Miscellaneous'] = h_tag.text.strip('\nMiscellaneous\n\n ').split("\n\n\n")
    
            guitar_list.append(guitar_dict)
    
            c_tag = div.find('img')
            # print(c_tag)
            # print(c_tag['src'])
    
            img_url = c_tag['src']
            img_resp = requests.get(img_url)
            # img_url
            # Ex. https://static.gibson.com/product-images/Custom/CUSFXC951/Cherry%20Teaburst/front-banner.png
            # imgresp.content
            p_filename = img_url.split('/')[-4:]
            filename = "".join(p_filename)
            # p_filename
            # filename
    
            guitar_picture_path = os.path.join(PICTURE_DIR, filename)  
            with open(guitar_picture_path, 'wb') as f:
                f.write(img_resp.content)
    
# 나중에 확인해 볼 코드 
# exec("{0} = crl.run('{0}')".format('Acoustic'))
# Acoustic
    
        print("총",len(guitar_list),"개의 결과물이 있습니다." )
        print("Gibson Accessory Kit의 주소는 https://www.gibson.com/Gear/Accessories/G-CAREKIT1 입니다.")
        return  guitar_list
    
# csv 쓰기 예시 
# Bass = crl.run("Bass")
# # Bass
# guitar_excel = Bass
# df = pd.DataFrame(guitar_excel)
# # df
# df.to_csv('gibsun_Bass.csv')

In [3]:
crl.run('Acoustic')

총 52 개의 결과물이 있습니다.
Gibson Accessory Kit의 주소는 https://www.gibson.com/Gear/Accessories/G-CAREKIT1 입니다.


[{'link': 'https://www.gibson.com/Guitar/ACC3NY964/125th-Anniversary-J-45',
  'name': '125th Anniversary J-45',
  'price': '$4,999.00',
  'Body': ['Shape\nRound Shoulder',
   'Top Material\nLutz Spruce',
   'Back Material\nFigured Mahogany',
   'Side Material\nFigured Mahogany',
   'Bracing\nTraditional hand-scalloped X-bracing',
   'Binding\nMulti-ply top, single-ply back',
   'Finish\nNitrocellulose'],
  'Neck': ['Material\nMahogany',
   'Neck Profile\nSlim Taper',
   'Scale Length\n24.75" / 62.865cm',
   'Fingerboard Material\nRosewood',
   'Fingerboard Radius\n12" / 304.8mm',
   'Number of Frets\n20',
   'Frets\nStandard',
   'Nut Material\nTusq',
   'Nut Width\n1.725" / 43.82mm',
   'Inlays\nMother-of-pearl dots, 125th headstock inlay'],
  'Hardware': ['Tuners\nGotoh open back',
   'Tuner Plating\nGold',
   'Truss Rod Cover\nLimited',
   'Pick Guard\nTear Drop',
   'Saddle Material\nTusq',
   'Bridge\nTraditional belly up, rosewood',
   'Bridge Pins\nTusq'],
  'Electronics': ['Con

# CSV 파일 쓰기

In [4]:
Bass = crl.run("Bass")
Bass

총 7 개의 결과물이 있습니다.
Gibson Accessory Kit의 주소는 https://www.gibson.com/Gear/Accessories/G-CAREKIT1 입니다.


[{'link': 'https://www.gibson.com/Guitar/USANRD503/EB-Bass-4-String-2019',
  'name': 'EB Bass 4 String 2019',
  'price': '$1,149.00',
  'Body': ['Material\nSwamp Ash',
   'Weight Relief\nNone',
   'Finish\nSatin Nitrocellulose Lacquer'],
  'Neck': ['Material\nMaple',
   'Neck Profile\nRounded',
   'Fingerboard Material\nRosewood',
   'Fingerboard Radius\n12" / 304.8mm',
   'Number of Frets\n24',
   'Frets\nMedium - Cryogenically Treated',
   'Nut Material\nTektoid',
   'Nut Width\n1.625" / 41.28mm',
   'End-of-Board Width\n2.484" / 63.09mm',
   'Inlays\nAcrylic Dots'],
  'Hardware': ['Finish\nChrome',
   'Bridge\nBABICZ Full Contact',
   'Tuners\nGrover Bass Keys',
   'Control Knobs\nKnurled Chrome Knob',
   'Jack Plate\nBlack'],
  'Electronics': ['Neck Pickup\nRhythm EB Bass',
   'Bridge Pickup\nLead EB Bass',
   'Controls\n2 Push/Pull Volumes (Coil Tap) & 1 Master T'],
  'Miscellaneous': ['Strings\n.045, .065, .085, .105',
   'Case\nDesigner Series Gig Bag',
   'Included Accessories\

In [5]:
guitar_excel = Bass
df = pd.DataFrame(guitar_excel)
# df

In [6]:
df.to_csv('gibsun_Bass.csv')

## 개선&보완점
#### 1. \n 제거 
#### 2. 태크 이름 변경
#### 3. 각 부품에 대한 한글설명서
#### 4. 엑셀 상에서의 순서변경
#### 5. 더 많은 사진